In [26]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

### FEtch Data from API

In [5]:
def get_alpha_vantage_news(api_key, ticker, topic=None):
    """
    Fetches news articles about `ticker` from the Alpha Vantage News API.
    Optionally filters by `topic` (e.g., "technology", "finance", etc.).
    Returns a list of dicts with article info (title, summary, sentiment, etc.).
    """

    # Alpha Vantage 'NEWS_SENTIMENT' endpoint
    url = "https://www.alphavantage.co/query"

    # Base params
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": ticker,
        "apikey": api_key,
        "sort": "RELEVANCE"
    }

    # If a topic was provided, include it in the params
    # NOTE: The Alpha Vantage parameter for filtering by topic is "topics".
    # E.g., topics=technology,ipo
    if topic:
        params["topics"] = topic

    response = requests.get(url, params=params)
    print("Request URL:", response.url)  # Debugging: see the final request
    data = response.json()

    # Check for errors
    if "feed" not in data:
        print(f"Error fetching news from Alpha Vantage: {data}")
        return []

    articles = data["feed"]
    return articles

In [6]:
def get_company_overview(api_key, ticker):
    """
    Fetches the company overview from Alpha Vantage using function=OVERVIEW.
    Returns a Pandas DataFrame.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "OVERVIEW",
        "symbol": ticker,
        "apikey": api_key
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "Symbol" not in data:
        print(f"Error fetching company overview: {data}")
        return None

    # Convert JSON dictionary to a Pandas DataFrame
    overview_df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])
    overview_df.reset_index(inplace=True)
    overview_df.columns = ['Field', 'Value']

    return overview_df

In [7]:
load_dotenv()
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")

In [8]:
articles = get_alpha_vantage_news(ALPHA_VANTAGE_API_KEY, "TSLA")

Request URL: https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=TSLA&apikey=4RZANL9BTFIUH6KF&sort=RELEVANCE


In [14]:
overview_data = get_company_overview(ALPHA_VANTAGE_API_KEY, "TSLA")

### Unstructured Loader

In [9]:
from langchain.document_loaders import UnstructuredURLLoader

In [10]:
def parse_article(articles):
    """
    Parses the article and returns the article text.
    """

    urls = []
    for i in range(len(articles)):
        urls.append(articles[i]['url'])

    loader = UnstructuredURLLoader(urls=urls)
    parsed_docs = loader.load()

    return parsed_docs

In [11]:
docs = parse_article(articles)

#### RecursiveTextSplitter

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
def split_text(docs):
    """
    Splits the text into smaller chunks.
    """

    text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', '.', ','],
    chunk_size=1000
    )

    chunks = text_splitter.split_documents(docs)
    texts = [chunk.page_content for chunk in chunks]

    return texts

In [16]:
chunks = split_text(docs)

In [17]:
len(chunks)

188

### FAISS Database

In [19]:
from sentence_transformers import SentenceTransformer
import faiss

In [20]:
def buildIndex(texts):
    """
    Build a FAISS index from a list of text documents.
    Each document is encoded into a vector using SentenceTransformer.
    The vectors are then normalized and added to the FAISS index.
    Args:
        text (list): List of text documents to be indexed.
    Returns:
        index (faiss.Index): A FAISS index containing the normalized vectors.
    """
    
    encoder = SentenceTransformer("BAAI/bge-base-en")
    vectors = encoder.encode(texts)
    faiss.normalize_L2(vectors)
    
    dim = vectors.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(vectors)

    return index


In [21]:
index = buildIndex(chunks)

In [22]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x29e41b810> >

In [55]:
# Save the FAISS index to a pickle file
import pickle

file_path = "faiss_store.pkl"
with open(file_path, "wb") as f:
    pickle.dump(index, f)

In [60]:
# Save the list to a file
with open("chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

In [61]:
# Load the list back
with open("chunks.pkl", "rb") as f:
    loaded_list = pickle.load(f)

In [ ]:
# Load
with open(file_path, "rb") as f:
    vectorstore = pickle.load(f)

### Retrival (Ignore)

In [23]:
search_query = "What's Tesla's latest model and price?"

In [25]:
encoder = SentenceTransformer("BAAI/bge-base-en")
vec = encoder.encode(search_query)
vec.shape

(768,)

In [27]:
svec = np.array(vec).reshape(1,-1) ##covert to same dim as source DB
svec.shape

(1, 768)

In [28]:
faiss.normalize_L2(svec)

In [30]:
distances, Indices = index.search(svec, k=10)

In [34]:
Indices.tolist()[0]

[66, 47, 31, 16, 32, 35, 20, 25, 58, 39]

In [42]:
for i in Indices.tolist()[0]:
    print(f"***********This is chunk: {i}", chunks[i])

***********This is chunk: 66 ▲ S&P 500 +---% | ▲ Stock Advisor +---% Join The Motley Fool

Accessibility Log In Help

Accessibility Menu

Free Article

You're reading a free article with opinions that may differ from The Motley Fool's Premium Investing Services. Become a Motley Fool member today to get instant access to our top analyst recommendations, in-depth research, investing resources, and more. Learn More

What's Going on With Tesla Stock?

By Parkev Tatevosian, CFA – Oct 29, 2024 at 5:05AM

NASDAQ: TSLA

Tesla

Market Cap

$898B

Today's Change

(-6.23%) -$17.39

Current Price

$261.71

Price as of March 6, 2025, 2:04 p.m. ET

A fantastic quarter has Tesla stock investors excited about the future.

Tesla (TSLA -6.23%) reported fantastic quarterly financial results, and the stock price is soaring in response.

Stock prices used were the afternoon prices of Oct. 24, 2024. The video was published on Oct. 26, 2024.
***********This is chunk: 47 ▲ S&P 500 +---% | ▲ Stock Advisor +---

In [58]:
from langchain.chains import RetrievalQAWithSourcesChain

In [67]:
from langchain import OpenAI

In [61]:
LLM_API_KEY = os.getenv("OPEN_ROUTER_KEY")

In [68]:
llm = OpenAI(temperature=0.9, max_tokens=500)

/var/folders/jj/1w9v7wf56_ldb_sbfv129l280000gn/T/ipykernel_80058/2582954192.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.9, max_tokens=500)


ValidationError: 1 validation error for OpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0.9, 'max...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [65]:
llm = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=LLM_API_KEY
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [59]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=index.as_retriever())

NameError: name 'llm' is not defined

In [1]:
BasePromptTemplate = 'Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION: Which state/country\'s law governs the interpretation of the contract?\n=========\nContent: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.\nSource: 28-pl\nContent: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.\n\n11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the continuation  in force of the remainder of the term (if any) and this Agreement.\n\n11.8 No Agency. Except as expressly stated otherwise, nothing in this Agreement shall create an agency, partnership or joint venture of any  kind between the parties.\n\n11.9 No Third-Party Beneficiaries.\nSource: 30-pl\nContent: (b) if Google believes, in good faith, that the Distributor has violated or caused Google to violate any Anti-Bribery Laws (as  defined in Clause 8.5) or that such a violation is reasonably likely to occur,\nSource: 4-pl\n=========\nFINAL ANSWER: This Agreement is governed by English law.\nSOURCES: 28-pl\n\nQUESTION: What did the president say about Michael Jackson?\n=========\nContent: Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland.\nSource: 0-pl\nContent: And we won’t stop. \n\nWe have lost so much to COVID-19. Time with one another. And worst of all, so much loss of life. \n\nLet’s use this moment to reset. Let’s stop looking at COVID-19 as a partisan dividing line and see it for what it is: A God-awful disease.  \n\nLet’s stop seeing each other as enemies, and start seeing each other for who we really are: Fellow Americans.  \n\nWe can’t change how divided we’ve been. But we can change how we move forward—on COVID-19 and other issues we must face together. \n\nI recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. \n\nThey were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. \n\nOfficer Mora was 27 years old. \n\nOfficer Rivera was 22. \n\nBoth Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. \n\nI spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves.\nSource: 24-pl\nContent: And a proud Ukrainian people, who have known 30 years  of independence, have repeatedly shown that they will not tolerate anyone who tries to take their country backwards.  \n\nTo all Americans, I will be honest with you, as I’ve always promised. A Russian dictator, invading a foreign country, has costs around the world. \n\nAnd I’m taking robust action to make sure the pain of our sanctions  is targeted at Russia’s economy. And I will use every tool at our disposal to protect American businesses and consumers. \n\nTonight, I can announce that the United States has worked with 30 other countries to release 60 Million barrels of oil from reserves around the world.  \n\nAmerica will lead that effort, releasing 30 Million barrels from our own Strategic Petroleum Reserve. And we stand ready to do more if necessary, unified with our allies.  \n\nThese steps will help blunt gas prices here at home. And I know the news about what’s happening can seem alarming. \n\nBut I want you to know that we are going to be okay.\nSource: 5-pl\nContent: More support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy. \n\nIn this Capitol, generation after generation, Americans have debated great questions amid great strife, and have done great things. \n\nWe have fought for freedom, expanded liberty, defeated totalitarianism and terror. \n\nAnd built the strongest, freest, and most prosperous nation the world has ever known. \n\nNow is the hour. \n\nOur moment of responsibility. \n\nOur test of resolve and conscience, of history itself. \n\nIt is in this moment that our character is formed. Our purpose is found. Our future is forged. \n\nWell I know this nation.\nSource: 34-pl\n=========\nFINAL ANSWER: The president did not mention Michael Jackson.\nSOURCES:\n\nQUESTION: {question}\n=========\n{summaries}\n=========\nFINAL ANSWER:'

In [3]:
print(BasePromptTemplate)

Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.

11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the con